In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cardiovascular_Disease"
cohort = "GSE283522"

# Input paths
in_trait_dir = "../../input/GEO/Cardiovascular_Disease"
in_cohort_dir = "../../input/GEO/Cardiovascular_Disease/GSE283522"

# Output paths
out_data_file = "../../output/preprocess/Cardiovascular_Disease/GSE283522.csv"
out_gene_data_file = "../../output/preprocess/Cardiovascular_Disease/gene_data/GSE283522.csv"
out_clinical_data_file = "../../output/preprocess/Cardiovascular_Disease/clinical_data/GSE283522.csv"
json_path = "../../output/preprocess/Cardiovascular_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Development and validation of a spatially informed assay that resolves biomarker discordance and predicts treatment response in breast cancer."
!Series_summary	"Background: Breast cancer (BCa) is a heterogeneous disease requiring precise diagnostics to guide effective treatment. Current assays fail to adequately address the complex biology of BCa subtypes/risk groups and accurately predict responses to treatments like antibody-drug conjugates (ADCs). To address these limitations, we developed and validated a novel diagnostic, prognostic, and predictive tool, mFISHseq."
!Series_summary	"Methods: Our approach, mFISHseq, integrates multiplexed RNA fluorescent in situ hybridization with RNA-sequencing, guided by laser capture microdissection. This technique ensures tumor purity, allows unbiased profiling of whole transcriptome data, and explicitly quantifies intratumoral heterogeneity."
!Series_summary	"Results: In a retrospective cohort study involvi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# After reviewing the background information, it appears the dataset contains RNA-seq data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For the cardiovascular disease trait, we need to identify a suitable surrogate from the sample characteristics
# Looking at the sample characteristics, there's no direct cardiovascular disease indicator,
# but we can infer from sample category in row 6 that this contains breast cancer data, not cardiovascular disease data
trait_row = None  # No cardiovascular disease data in this dataset

# Age data is available in row 2
age_row = 2

# Gender/Sex data is available in row 5
gender_row = 5

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> Optional[int]:
    """
    This function is not used as trait_row is None, but we need to define it
    for the geo_select_clinical_features function parameter.
    """
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age ranges to numerical values (midpoint of the range)."""
    if value is None or pd.isna(value) or 'not applicable' in str(value).lower() or 'missing' in str(value).lower():
        return None
    
    # Extract the age value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Handle age ranges like "55 - 59"
    if '-' in value:
        try:
            age_range = value.strip().split('-')
            lower_bound = int(age_range[0].strip())
            upper_bound = int(age_range[1].strip())
            return (lower_bound + upper_bound) / 2
        except (ValueError, IndexError):
            return None
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (0 for female, 1 for male)."""
    if value is None or pd.isna(value) or 'not applicable' in str(value).lower() or 'missing' in str(value).lower():
        return None
    
    # Extract the gender value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    return None

# 3. Save Metadata
# Conduct initial filtering on usability of the dataset
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=(trait_row is not None)
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we're going to skip this substep
# No need to execute geo_select_clinical_features or save clinical data


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")
print(f"SOFT file found: {soft_file}")

# 2. Implement a diagnostic approach to examine the file structure first
try:
    # First look at the file structure to understand how to extract data correctly
    with gzip.open(matrix_file, 'rt') as file:
        # Read first 50 lines to check file format
        first_lines = [file.readline().strip() for _ in range(50)]
        print("First few lines of the file:")
        for i, line in enumerate(first_lines[:10]):  # Print just 10 lines to avoid overwhelming output
            print(f"{i}: {line[:100]}...")  # Truncate long lines
        
        # Check for matrix markers by scanning the whole file
        file.seek(0)  # Reset to beginning of file
        has_begin_marker = False
        has_end_marker = False
        line_count = 0
        for line in file:
            line_count += 1
            if '!series_matrix_table_begin' in line:
                has_begin_marker = True
                print(f"Found begin marker at line {line_count}")
            elif '!series_matrix_table_end' in line:
                has_end_marker = True
                print(f"Found end marker at line {line_count}")
                
        print(f"Total lines in file: {line_count}")
        print(f"Has begin marker: {has_begin_marker}")
        print(f"Has end marker: {has_end_marker}")
    
    # 3. Try to read the gene expression data from the SOFT file instead
    print("\nAttempting to read gene data from SOFT file:")
    gene_data = get_gene_annotation(soft_file)
    print(f"Gene data from SOFT file shape: {gene_data.shape}")
    
    # 4. Print the first 20 gene identifiers (if available)
    if gene_data.shape[0] > 0:
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20])
    else:
        print("No gene identifiers found in the SOFT file.")
    
    # 5. Check column headers to understand data structure
    print("\nFirst 5 column headers:")
    print(gene_data.columns[:5])

except Exception as e:
    print(f"Error processing files: {e}")
    import traceback
    traceback.print_exc()


Matrix file found: ../../input/GEO/Cardiovascular_Disease/GSE283522/GSE283522_series_matrix.txt.gz
SOFT file found: ../../input/GEO/Cardiovascular_Disease/GSE283522/GSE283522_family.soft.gz
First few lines of the file:
0: !Series_title	"Development and validation of a spatially informed assay that resolves biomarker disc...
1: !Series_geo_accession	"GSE283522"...
2: !Series_status	"Public on Dec 23 2024"...
3: !Series_submission_date	"Dec 04 2024"...
4: !Series_last_update_date	"Dec 23 2024"...
5: !Series_summary	"Background: Breast cancer (BCa) is a heterogeneous disease requiring precise diagno...
6: !Series_summary	"Methods: Our approach, mFISHseq, integrates multiplexed RNA fluorescent in situ hyb...
7: !Series_summary	"Results: In a retrospective cohort study involving 1,082 FFPE breast tumors, mFISHs...
8: !Series_summary	"Conclusion: The mFISHseq method solves a long-standing challenge in the precise dia...
9: !Series_overall_design	"Out of a starting cohort of 1,082 breast samp

Traceback (most recent call last):
  File "/tmp/ipykernel_59763/726718462.py", line 36, in <module>
    gene_data = get_gene_annotation(soft_file)
  File "/media/techt/DATA/GenoAgent/tools/preprocess.py", line 123, in get_gene_annotation
    gene_metadata = filter_content_by_prefix(file_path, prefixes_a=prefixes, unselect=True, source_type='file',
  File "/media/techt/DATA/GenoAgent/tools/preprocess.py", line 97, in filter_content_by_prefix
    filtered_content_a = pd.read_csv(io.StringIO(filtered_content_a), delimiter='\t', low_memory=False,
  File "/home/techt/anaconda3/envs/agent/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/home/techt/anaconda3/envs/agent/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/home/techt/anaconda3/envs/agent/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 16

### Step 4: Gene Identifier Review

In [5]:
# Let's analyze the available information to determine if gene mapping is needed

# From the provided information, we can see the Platform ID is GPL24676
# The platform ID is a crucial piece of information for determining whether gene mapping is needed

# GPL24676 is an Illumina platform that typically uses standard gene symbols rather than probe IDs
# This platform is often used for targeted RNA-seq applications and multiplexed RNA fluorescent 
# in situ hybridization with sequencing (mFISHseq) as mentioned in the study description

# Based on this platform information, it's likely that the data already uses standardized gene symbols
# and does not require mapping to human gene symbols

# For Illumina GPL24676 platform, gene identifiers are typically already in standard 
# human gene symbol format

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Try to load the gene expression data using get_genetic_data instead of get_gene_annotation
try:
    # Get the SOFT and matrix file paths again 
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    print(f"Matrix file found: {matrix_file}")
    
    # Examine the matrix file structure
    with gzip.open(matrix_file, 'rt') as file:
        print("Checking file structure...")
        has_begin_marker = False
        begin_marker_line = 0
        
        for i, line in enumerate(file):
            if '!series_matrix_table_begin' in line:
                has_begin_marker = True
                begin_marker_line = i
                print(f"Found begin marker at line {i}")
                # Print next few lines to debug
                print("Lines after begin marker:")
                for j in range(3):
                    next_line = file.readline().strip()
                    print(f"  {next_line[:100]}...")
                break
                
        print(f"Begin marker found: {has_begin_marker}")
    
    # Attempt to read gene expression data properly
    try:
        gene_data = get_genetic_data(matrix_file)
        print(f"Original gene data shape: {gene_data.shape}")
    except Exception as e:
        print(f"Standard get_genetic_data failed: {e}")
        print("Trying alternative approach...")
        
        # Alternative approach: manually read the data
        with gzip.open(matrix_file, 'rt') as file:
            # Skip to the line after !series_matrix_table_begin
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Read the header (sample IDs)
            header = file.readline().strip().split('\t')
            
            # Read the data rows
            rows = []
            for line in file:
                if '!series_matrix_table_end' in line:
                    break
                rows.append(line.strip().split('\t'))
            
            # Create DataFrame
            if rows:
                gene_data = pd.DataFrame(rows, columns=header)
                gene_data = gene_data.rename(columns={gene_data.columns[0]: 'ID'})
                gene_data = gene_data.set_index('ID')
                print(f"Manually extracted gene data shape: {gene_data.shape}")
            else:
                print("No data rows found between begin and end markers")
                gene_data = pd.DataFrame()
    
    # Print the first few rows to understand the data structure
    if not gene_data.empty:
        print("First few rows of gene data:")
        print(gene_data.head(2))
    
except Exception as e:
    print(f"Error extracting gene data: {e}")
    # Create an empty DataFrame if extraction fails
    gene_data = pd.DataFrame()
    print("Created empty gene data DataFrame due to extraction failure")

# Since trait_row was None in step 2, we don't have any cardiovascular disease trait data in this dataset
print("No cardiovascular disease trait data available in this dataset")

# Check if gene data is available and has a valid structure
if not gene_data.empty:
    try:
        # 1. Normalize gene symbols
        gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
        print(f"Normalized gene data shape: {gene_data_normalized.shape}")
        
        # Save the gene expression data 
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        gene_data_normalized.to_csv(out_gene_data_file)
        print(f"Gene expression data saved to {out_gene_data_file}")
    except Exception as e:
        print(f"Gene normalization failed: {e}")
        gene_data_normalized = pd.DataFrame()
        print("Created empty normalized gene data DataFrame due to normalization failure")
else:
    print("Gene data unavailable or empty, skipping normalization")
    gene_data_normalized = pd.DataFrame()

# Since we don't have trait data (trait_row is None), we create an empty clinical DataFrame
clinical_features = pd.DataFrame()
print("Created empty clinical features DataFrame as no trait data available")

# No clinical data to save
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
empty_df = pd.DataFrame()
empty_df.to_csv(out_clinical_data_file)
print(f"Empty clinical data saved to {out_clinical_data_file}")

# Since we don't have trait data, we cannot create valid linked data
linked_data = pd.DataFrame({'placeholder': [0]})  # Create minimal DataFrame for validation
print("No valid linked data can be created due to missing trait information")

# Conduct final quality validation - dataset is not usable due to missing trait information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=(not gene_data.empty),
    is_trait_available=False,  # No trait data available
    is_biased=False,  # Providing a boolean value as required
    df=linked_data,  # Providing minimal valid DataFrame
    note="Dataset contains breast cancer data, not cardiovascular disease data. No trait information available for the requested trait."
)

print(f"Dataset usability status: {is_usable}")
print("Dataset deemed not usable for cardiovascular disease associative studies. No linked data saved.")

Matrix file found: ../../input/GEO/Cardiovascular_Disease/GSE283522/GSE283522_series_matrix.txt.gz
Checking file structure...
Found begin marker at line 205
Lines after begin marker:
  "ID_REF"	"GSM8664605"	"GSM8664606"	"GSM8664607"	"GSM8664608"	"GSM8664609"	"GSM8664610"	"GSM8664611"	...
  !series_matrix_table_end...
  ...
Begin marker found: True


Original gene data shape: (0, 1254)
No cardiovascular disease trait data available in this dataset
Gene data unavailable or empty, skipping normalization
Created empty clinical features DataFrame as no trait data available
Empty clinical data saved to ../../output/preprocess/Cardiovascular_Disease/clinical_data/GSE283522.csv
No valid linked data can be created due to missing trait information
Abnormality detected in the cohort: GSE283522. Preprocessing failed.
Dataset usability status: False
Dataset deemed not usable for cardiovascular disease associative studies. No linked data saved.
